In [1]:
from pandas import DataFrame


In [12]:
data = {
    "Teams":["MANCITY", "MANUTD","LIVERPOOL","CHELSEA","SPURS","ARSENAL","LEICEISTER","WESTHAM","ASTONVILLA","CRYSTALPALACE","BRIGHTON","EVERTON","FULHAM","BOURNEMOTH"],
    "points":[94,88,86,80,79,73,72,68,64,61,58,53,49,44],
    "G.D" :[86,78,77,70,67,65,66,56,45,33,31,23,22,34]
}

LeagueTable = DataFrame(data,index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14])

In [13]:
print(LeagueTable)

            Teams  points  G.D
1         MANCITY      94   86
2          MANUTD      88   78
3       LIVERPOOL      86   77
4         CHELSEA      80   70
5           SPURS      79   67
6         ARSENAL      73   65
7      LEICEISTER      72   66
8         WESTHAM      68   56
9      ASTONVILLA      64   45
10  CRYSTALPALACE      61   33
11       BRIGHTON      58   31
12        EVERTON      53   23
13         FULHAM      49   22
14     BOURNEMOTH      44   34


In [16]:
LeagueTable.tail(4)

,Teams,points,G.D
11,BRIGHTON,58,31
12,EVERTON,53,23
13,FULHAM,49,22
14,BOURNEMOTH,44,34


In [17]:
LeagueTable["Teams"]

1           MANCITY
2            MANUTD
3         LIVERPOOL
4           CHELSEA
5             SPURS
6           ARSENAL
7        LEICEISTER
8           WESTHAM
9        ASTONVILLA
10    CRYSTALPALACE
11         BRIGHTON
12          EVERTON
13           FULHAM
14       BOURNEMOTH
Name: Teams, dtype: object

In [18]:
LeagueTable.loc[5]

Teams     SPURS
points       79
G.D          67
Name: 5, dtype: object

In [19]:
LeagueTable


,Teams,points,G.D
1,MANCITY,94,86
2,MANUTD,88,78
3,LIVERPOOL,86,77
4,CHELSEA,80,70
5,SPURS,79,67
6,ARSENAL,73,65
7,LEICEISTER,72,66
8,WESTHAM,68,56
9,ASTONVILLA,64,45
10,CRYSTALPALACE,61,33


In [32]:
a =3
b =9
a+b 

12

In [33]:
a>b

False

In [34]:
# Circle CI configuration file
# https://circleci.com/docs/

version: 2.1


#######################################
# Define some common steps as commands.
#

commands:
  check-skip:
    steps:
      - run:
          name: Check-skip
          command: |
            export git_log=$(git log --max-count=1 --pretty=format:"%B" | tr "\n" " ")
            echo "Got commit message:"
            echo "${git_log}"
            if [[ -v CIRCLE_PULL_REQUEST ]] && ([[ "$git_log" == *"[skip circle]"* ]] || [[ "$git_log" == *"[circle skip]"* ]]); then
              echo "Skip detected, exiting job ${CIRCLE_JOB} for PR ${CIRCLE_PULL_REQUEST}."
              circleci-agent step halt;
            fi
  merge:
    steps:
      - run:
          name: Merge with upstream
          command: |
            if ! git remote -v | grep upstream; then
              git remote add upstream https://github.com/matplotlib/matplotlib.git
            fi
            git fetch upstream
            if [[ "$CIRCLE_BRANCH" != "main" ]] && \
               [[ "$CIRCLE_PR_NUMBER" != "" ]]; then
              echo "Merging ${CIRCLE_PR_NUMBER}"
              git pull --ff-only upstream "refs/pull/${CIRCLE_PR_NUMBER}/merge"
            fi
  apt-install:
    steps:
      - run:
          name: Install apt packages
          command: |
            sudo apt -qq update
            sudo apt install -y \
              inkscape \
              ffmpeg \
              dvipng \
              lmodern \
              cm-super \
              texlive-latex-base \
              texlive-latex-extra \
              texlive-fonts-recommended \
              texlive-latex-recommended \
              texlive-pictures \
              texlive-xetex \
              ttf-wqy-zenhei \
              graphviz \
              fonts-crosextra-carlito \
              fonts-freefont-otf \
              fonts-humor-sans \
              fonts-noto-cjk \
              optipng
  fonts-install:
    steps:
      - restore_cache:
          key: fonts-2
      - run:
          name: Install custom fonts
          command: |
            mkdir -p ~/.local/share/fonts
            wget -nc https://github.com/google/fonts/blob/master/ofl/felipa/Felipa-Regular.ttf?raw=true -O ~/.local/share/fonts/Felipa-Regular.ttf || true
            fc-cache -f -v
      - save_cache:
          key: fonts-2
          paths:
            - ~/.local/share/fonts/

  pip-install:
    description: Upgrade pip and setuptools and wheel to get as clean an install as possible
    steps:
      - run:
          name: Upgrade pip, setuptools, wheel
          command: |
            python -m pip install --upgrade --user pip
            python -m pip install --upgrade --user wheel
            python -m pip install --upgrade --user 'setuptools!=60.6.0'
  doc-deps-install:
    parameters:
      numpy_version:
        type: string
        default: ""
    steps:
      - run:
          name: Install Python dependencies
          command: |
            python -m pip install --no-deps --user \
                git+https://github.com/matplotlib/mpl-sphinx-theme.git
            python -m pip install --user \
                numpy<< parameters.numpy_version >> codecov coverage \
                -r requirements/doc/doc-requirements.txt
  mpl-install:
    steps:
      - run:
          name: Install Matplotlib
          command: |
            if [[ "$CIRCLE_BRANCH" == v*-doc ]]; then
              # The v*-doc branches must build against the specified release.
              version=${CIRCLE_BRANCH%-doc}
              version=${version#v}
              python -m pip install matplotlib==${version}
            else
              python -m pip install --user -ve .
            fi
      - save_cache:
          key: build-deps-1
          paths:
            # FreeType 2.6.1 tarball.
            - ~/.cache/matplotlib/0a3c7dfbda6da1e8fce29232e8e96d987ababbbf71ebc8c75659e4132c367014
            # Qhull 2020.2 tarball.
            - ~/.cache/matplotlib/b5c2d7eb833278881b952c8a52d20179eab87766b00b865000469a45c1838b7e

  doc-build:
    steps:
      - restore_cache:
          keys:
            - sphinx-env-v1-{{ .BuildNum }}-{{ .Environment.CIRCLE_JOB }}
            - sphinx-env-v1-{{ .Environment.CIRCLE_PREVIOUS_BUILD_NUM }}-{{ .Environment.CIRCLE_JOB }}
      - run:
          name: Build documentation
          command: |
            # Set epoch to date of latest tag.
            export SOURCE_DATE_EPOCH="$(git log -1 --format=%at $(git describe --abbrev=0))"
            # Set release mode only when deploying to devdocs.
            if [ "$CIRCLE_PROJECT_USERNAME" = "matplotlib" ] && \
               [ "$CIRCLE_BRANCH" = "main" ] && \
               [ "$CIRCLE_PR_NUMBER" = "" ]; then
              export RELEASE_TAG='-t release'
            fi
            make html O="-T $RELEASE_TAG"
            rm -r build/html/_sources
          working_directory: doc
      - save_cache:
          key: sphinx-env-v1-{{ .BuildNum }}-{{ .Environment.CIRCLE_JOB }}
          paths:
            - doc/build/doctrees

  doc-bundle:
    steps:
      - run:
          name: Bundle sphinx-gallery documentation artifacts
          command: tar cf doc/build/sphinx-gallery-files.tar.gz doc/api/_as_gen doc/gallery doc/tutorials
          when: always
      - store_artifacts:
          path: doc/build/sphinx-gallery-files.tar.gz


##########################################
# Here is where the real jobs are defined.
#

jobs:
  docs-python38:
    docker:
      - image: cimg/python:3.8
    steps:
      - checkout
      - check-skip
      - merge

      - apt-install
      - fonts-install
      - pip-install

      - mpl-install
      - doc-deps-install

      - doc-build

      - doc-bundle

      - store_artifacts:
          path: doc/build/html
      - store_test_results:
          path: doc/build/test-results

      - add_ssh_keys:
          fingerprints:
            - "6b:83:76:a5:7d:bd:ce:19:a4:e3:81:e0:80:16:a4:fe"
      - deploy:
          name: "Deploy new docs"
          command: ./.circleci/deploy-docs.sh

#########################################
# Defining workflows gets us parallelism.
#

workflows:
  version: 2
  build:
    jobs:
      - docs-python38

SyntaxError: invalid syntax (<ipython-input-34-74a5d0a0d2de>, line 11)

In [35]:
import ttk

ModuleNotFoundError: No module named 'ttk'

In [36]:
import tkinter as tk
from tkinter import ttk